In [5]:
# Reset the schema
import duckdb

# Connect to your DuckDB database (file or in-memory)
conn = duckdb.connect("mtga_local.duckdb")  # or ':memory:'

# Read the SQL file
with open("silver_duckdb_schema.sql", "r") as f:
    sql_script = f.read()

# Execute the entire SQL script
conn.execute(sql_script)

# Optional: check tables
tables = conn.execute("SHOW TABLES").fetchall()
conn.close()
print(tables)


[('decks',), ('dim_cards',), ('matches',), ('players',), ('turn1_hands',)]


In [4]:
# Load the dim_cards table
import pandas as pd
import duckdb
import json

# 1. Read CSV safely
df = pd.read_csv(
    '/home/r3gal/develop/mtga_pipeline/data/references/dim_cards.csv',
    dtype=str,      # read all columns as strings
    keep_default_na=False  # keep empty fields as empty strings instead of NaN if desired
)

# Optional: Convert JSON/list columns from strings to JSON strings
json_cols = ['colors', 'color_identity', 'legalities']
for col in json_cols:
    df[col] = df[col].apply(lambda x: x if x.startswith('[') or x.startswith('{') else '[]')

df = df.drop_duplicates(keep='first', subset='arena_id')
# 2. Connect to DuckDB
conn = duckdb.connect("mtga_local.duckdb")

# 3. Register the dataframe and insert into dim_cards
conn.register("df_temp", df)
conn.execute("INSERT INTO dim_cards SELECT * FROM df_temp")

# 4. Preview
# print(conn.execute("SELECT * FROM dim_cards LIMIT 5").fetchall())
conn.close()


In [ ]:

import pandas as pd
import duckdb
from pprint import pprint
import ast

# csv_path = "/home/r3gal/develop/mtga_pipeline/cloud/test_4mulligan.csv"
csv_path = "/home/r3gal/develop/mtga_pipeline/cloud/test_3.csv"
# csv_path = "/home/r3gal/develop/mtga_pipeline/cloud/test.csv"
df = pd.read_csv(csv_path)


conn = duckdb.connect("mtga_local.duckdb")  # or ':memory:'
# conn.close()

In [87]:
# insert matches
# CREATE TABLE matches (
#     match_id BIGINT PRIMARY KEY,
#     deck_id BIGINT NOT NULL,
#     player_id VARCHAR,
#     player_seat INTEGER, -- 1 or 2
#     start_time TIMESTAMP NOT NULL,
#     duration INTEGER,               -- SECONDS
#     winner_seat VARCHAR,  -- 1 or 2
#     format VARCHAR,
#     draw_order VARCHAR
# );

import pandas as pd
import duckdb
import json
from pprint import pprint
import ast

# csv_path = "/home/r3gal/develop/mtga_pipeline/cloud/test_4mulligan.csv"
# csv_path = "/home/r3gal/develop/mtga_pipeline/cloud/test_3.csv"
csv_path = "/home/r3gal/develop/mtga_pipeline/cloud/test.csv"
df = pd.read_csv(csv_path)

# drop rows where game_num != 1 for testing

conn = duckdb.connect("mtga_local.duckdb")  # or ':memory:'
# import ast
df['payload'] = df['payload'].apply(ast.literal_eval)

# intake = conn, df

match_id = conn.execute("SELECT COALESCE(MAX(match_id), 0) FROM matches").fetchone()[0] + 1
player_id = df.iloc[0]['player_id']
# comment below in once in main fucntion
# deck_id = insert_deck(conn, df, match_id)
deck_id = 2

df['timestamp_f'] =  pd.to_datetime(
    df['timestamp'],
    format='%m/%d/%Y %I:%M:%S %p'
    )
start_time = df.iloc[0]['timestamp_f']

duration = df.iloc[-1]['timestamp_f'] - df.iloc[0]['timestamp_f']
duration_seconds = int(duration.total_seconds())

attributes = json.loads(df['payload'].iloc[0]['request']).get('Summary').get('Attributes')
game_format = next(
    (attr['value'] for attr in attributes if attr['name'] == 'Format'),
    None
)

player_seat = 0
players = df['payload'].iloc[-1].get('gameRoomConfig').get('reservedPlayers')
# print(players)
for item in players:
    if item.get('userId') == player_id:
        player_seat = item.get('systemSeatId')

# winner_seat
# 'MatchScope_Game' -> Is the result of one game in a match (can be 1 or 3 games per match)
match_results = df['payload'].iloc[-1].get('finalMatchResult').get('resultList')
winner_seat = 0
for item in match_results:
    if item.get('scope') == 'MatchScope_Match':
        winner_seat = item.get('winningTeamId')

# draw_order ??
#   extra goal


# will change draw_order after implementation
draw_order = ''
conn.execute(
    """
    INSERT INTO matches (match_id, deck_id, player_id, player_seat, start_time, duration, winner_seat, game_format, draw_order)
    SELECT ?, ?, ?, ?, ?, ?, ?, ?, ?
    """,
    (int(match_id), int(deck_id), str(player_id), int(player_seat), start_time, int(duration_seconds), int(winner_seat), str(game_format), str(draw_order))
)


In [1]:
import duckdb

%load_ext sql
%sql duckdb:///mtga_local.duckdb

In [2]:
%%sql
SELECT 
(SELECT count(*) FROM matches) as matches_rows,
(SELECT count(*) FROM decks) as decks_rows,
(SELECT count(*) FROM turn1_hands) as t1hand_rows,
(SELECT count(*) FROM dim_cards) as cards_rows
;

 * duckdb:///mtga_local.duckdb
Done.


matches_rows,decks_rows,t1hand_rows,cards_rows
147,146,203,16434


In [7]:
%%sql
select * from matches limit 5; 

 * duckdb:///mtga_local.duckdb
Done.


match_id,deck_id,player_id,player_seat,start_time,duration,winner_seat,game_format,draw_order
1,1,V7JT5YS7ANCWRHG35IZNV4PKOY,1,2026-02-22 20:10:36,278,2,Standard,
2,2,V7JT5YS7ANCWRHG35IZNV4PKOY,1,2026-02-22 20:15:41,113,2,Standard,
3,3,V7JT5YS7ANCWRHG35IZNV4PKOY,1,2026-02-16 13:51:43,294,2,Standard,
4,4,V7JT5YS7ANCWRHG35IZNV4PKOY,1,2026-02-16 13:57:14,219,1,Standard,
5,5,V7JT5YS7ANCWRHG35IZNV4PKOY,2,2026-02-12 21:58:32,227,2,Standard,


In [8]:
%%sql 
SELECT h.* 
FROM turn1_hands h
join matches m
on h.match_id = m.match_id
where h.mulliganCount > 0
and m.player_seat = m.winner_seat;


 * duckdb:///mtga_local.duckdb
Done.


hand_id,player_id,match_id,initial_hand,mulliganCount,final_hand,went_first
5,V7JT5YS7ANCWRHG35IZNV4PKOY,4,"[90793, 90793, 95949, 87246, 95614, 92199]",1,[90793],True
8,V7JT5YS7ANCWRHG35IZNV4PKOY,6,"[90793, 90793, 91644, 96696, 95614, 92199]",1,[91654],True
12,V7JT5YS7ANCWRHG35IZNV4PKOY,9,"[90793, 90793, 95949, 67868, 96696, 69548]",1,[91644],True
25,V7JT5YS7ANCWRHG35IZNV4PKOY,19,"[90794, 90794, 90794, 95949, 92199, 92199]",1,[87231],False
66,V7JT5YS7ANCWRHG35IZNV4PKOY,46,"[90794, 90794, 90794, 90434, 91644, 91631]",1,[91644],True
68,V7JT5YS7ANCWRHG35IZNV4PKOY,47,"[95192, 95192, 95200, 93729, 93729, 93715]",1,[94007],True
96,V7JT5YS7ANCWRHG35IZNV4PKOY,68,"[90793, 90793, 90793, 90434, 91619, 90452]",1,[91619],False
107,V7JT5YS7ANCWRHG35IZNV4PKOY,75,"[93846, 90793, 91644, 91644, 87246, 95614]",1,[95614],True
118,V7JT5YS7ANCWRHG35IZNV4PKOY,81,"[93846, 90793, 90793, 67868, 97369, 95614]",1,[67868],False
176,V7JT5YS7ANCWRHG35IZNV4PKOY,127,"[90794, 90794, 95949, 67868, 90434, 92199]",1,[90794],False
